## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-02-14-55-15 +0000,nyt,Research Library at NASA’s Goddard Space and F...,https://www.nytimes.com/2025/12/31/climate/nas...
1,2026-01-02-14-48-42 +0000,nyt,Trump Says U.S. Is ‘Locked and Loaded’ if Iran...,https://www.nytimes.com/2026/01/02/world/middl...
2,2026-01-02-14-44-15 +0000,startribune,Eight players with Minnesota ties named to U.S...,https://www.startribune.com/eight-players-with...
3,2026-01-02-14-43-00 +0000,wsj,Oil Futures Open Year Lower on Oversupply Conc...,https://www.wsj.com/articles/oil-rises-prices-...
4,2026-01-02-14-39-14 +0000,nyt,Live Updates: Families of Missing Plead for An...,https://www.nytimes.com/live/2026/01/02/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2500/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,new,31
28,year,31
9,trump,23
37,swiss,21
39,fire,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
90,2026-01-02-08-05-41 +0000,nypost,Swiss face painful task of identifying victims...,https://nypost.com/2026/01/02/world-news/swiss...,135
149,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,134
191,2026-01-01-17-09-57 +0000,nypost,Foreign tourists among 47 confirmed dead in Ne...,https://nypost.com/2026/01/01/world-news/many-...,118
138,2026-01-02-01-34-44 +0000,nyt,Fire in Swiss Alps Leaves Dozens of New Year’s...,https://www.nytimes.com/2026/01/01/world/europ...,116
4,2026-01-02-14-39-14 +0000,nyt,Live Updates: Families of Missing Plead for An...,https://www.nytimes.com/live/2026/01/02/world/...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
90,135,2026-01-02-08-05-41 +0000,nypost,Swiss face painful task of identifying victims...,https://nypost.com/2026/01/02/world-news/swiss...
85,56,2026-01-02-08-56-35 +0000,nypost,Trump vows US ‘locked and loaded’ to help Iran...,https://nypost.com/2026/01/02/world-news/trump...
48,34,2026-01-02-11-48-12 +0000,cbc,Kim Jong-un's daughter visits family mausoleum...,https://www.cbc.ca/news/world/north-korea-heir...
172,31,2026-01-01-19-41-07 +0000,nypost,Swiss nightclub where 47 people died advertise...,https://nypost.com/2026/01/01/world-news/swiss...
191,30,2026-01-01-17-09-57 +0000,nypost,Foreign tourists among 47 confirmed dead in Ne...,https://nypost.com/2026/01/01/world-news/many-...
36,29,2026-01-02-12-27-57 +0000,nypost,Mayor Mamdani blasted for revoking city polici...,https://nypost.com/2026/01/02/us-news/nyc-mayo...
87,25,2026-01-02-08-43-34 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...
142,25,2026-01-02-00-31-50 +0000,nypost,"Terrifying video, cockpit audio captures momen...",https://nypost.com/2026/01/01/us-news/terrifyi...
52,24,2026-01-02-11-16-28 +0000,nypost,Young golfer Emanuele Galeppini named as first...,https://nypost.com/2026/01/02/world-news/golfe...
34,23,2026-01-02-12-43-02 +0000,nypost,"Trump doubles down, insists he is in ‘perfect ...",https://nypost.com/2026/01/02/us-news/trump-do...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
